## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
#from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all

from functions.aux import MetricTotalAccuraccy
# Load tree transformation
from functions.transform_inference import TransformerInference

from net import NeoNet

class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


IS CUDA AVAILABLE: True


131072

### Basic parameters

In [2]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.2_subqueries'
model_name = 'Model_Adam'
model_path = "./" + model_name + '.pt'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"
optimizer = "Adam"
#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
#min_time_or = 15
#max_time_or = 80
#percent_of_data = 1
#min_time = 5
#max_time = 80


In [3]:
epsilon = 0.75 ## EPSILON
lr_rl = 0.9 ## LEARNING RATE RL
disc_fac = 0.9 # DISCOUNT FACTOR RL 
epoch_rl = 200 ### TOTAL EPOCH POR DATA RL
n = "_" + model_name + "_gamma_" + str(disc_fac) + "_"##SALT Nombre archivos

### Features

In [4]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

In [5]:
ds_train = pd.read_csv(URL + csv_name + '_ds_train_ft.csv', engine='python', encoding='utf-8')
ds_val = pd.read_csv(URL + csv_name + '_ds_val_ft.csv', engine='python', encoding='utf-8')
ds_test = pd.read_csv(URL + csv_name + '_ds_test_ft.csv', engine='python', encoding='utf-8')
ds_rl = pd.read_csv(URL + csv_name + '_ds_rl_ft.csv', engine='python', encoding='utf-8')

In [6]:
print("ds_train.shape",ds_train.shape)
print("ds_val.shape",ds_val.shape)
print("ds_test.shape",ds_test.shape)
print("ds_rl.shape",ds_rl.shape)

ds_train.shape (7938, 15)
ds_val.shape (1641, 15)
ds_test.shape (1072, 15)
ds_rl.shape (2000, 67)


# FUNCTIONS

In [7]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [8]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

Size Train: 7938, Val 1641


In [9]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [10]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (7938, 15)
shape ds_val  : (1641, 15)
shape ds_test : (1072, 15)
shape ds_rl : (2000, 67)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (1641, 5)
shape x_train_query: (7938, 5)
----------x_plan_level_data----------
shape x_val_tree  : (1641,)
shape x_train_tree: (7938,)
----------y_data------------
shape y_val  : (1641,)
shape y_train: (7938,)

----------TEST DATA----------
shape x_test_tree : (1072,)
shape x_test_query: (1072, 5)
shape y_test      : (1072,)
-----------------------

----------RL DATA----------
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


In [11]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

0

In [12]:
data_inference = {}
bao_base = ""
with (open(bao_base + "preprocess_inference_info_" + model_name + ".pickle", "rb")) as openfile:
    while True:
        try:
            data_inference = pickle.load(openfile)
        except EOFError:
            break
# Load Scaler for query features.
scalerx = None
with (open(bao_base + "scalerx_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            scalerx = pickle.load(openfile)
        except EOFError:
            break

pipeline_inverse = None
with (open(bao_base + "pipeline_inverse_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            pipeline_inverse = pickle.load(openfile)
        except EOFError:
            break
with open('NeoNetValues_' + model_name +'.txt', 'r') as file:
    data = file.read().rstrip()
neo_data = ast.literal_eval(data)

IS CUDA AVAILABLE: True


In [13]:
def ActivationFuncReluOrLeaky(string):
    if string == "<class 'torch.nn.modules.activation.LeakyReLU'>":
        return nn.LeakyReLU
    elif string == "<class 'torch.nn.modules.activation.ReLU'>":
        return nn.ReLU
io_dim_model_data = int(neo_data['io_dim'])
query_input_size = int(neo_data['query_input_size'])
query_hidden_inputs = ast.literal_eval(neo_data['query_hidden_inputs'])
query_output = int(neo_data['query_output'])
tree_units = ast.literal_eval(neo_data['tree_units'])
tree_units_dense = ast.literal_eval(neo_data['tree_units_dense'])
tree_activation_tree = ActivationFuncReluOrLeaky(neo_data['activation_tree'])
tree_activation_dense = ActivationFuncReluOrLeaky(neo_data['activation_dense'])

def get_prediction_dataset_from_load(ds, list_columns):
    x_tree = ds["trees"].values
    x_query = ds[list_columns]
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    targets = list(ds["time"].values)
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}

def get_prediction_from_load(x_tree, x_query, targets):
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}


In [14]:
reg = NeoNet(
    io_dim_model_data,
    query_input_size,
    query_hidden_inputs,
    query_output=query_output,
    tree_units=tree_units,
    tree_units_dense=tree_units_dense,
    activation_tree=tree_activation_tree,
    activation_dense=tree_activation_dense,
)
device = torch.device("cuda")
# Loading model parameters.
reg_path = model_path
#reg.load_state_dict(torch.load(reg_path, map_location=torch.device('cpu')))
reg.load_state_dict(torch.load(reg_path))
reg.to(device)
transformer = TransformerInference(
        model=reg,
        tree_transform=data_inference["tree_transform"],
        pipeline_inverse=pipeline_inverse,
        scalerx=scalerx,
        maxcardinality=data_inference["maxcardinality"]
    )
print("Modelos Listos")

Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(1353, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias=True)]
Modelos Listos


In [15]:
 ds_test['time']

0         1.687920
1         1.678266
2         1.109980
3         1.858232
4         1.598426
           ...    
1067    107.000000
1068    101.000000
1069    123.000000
1070    120.000000
1071    138.003726
Name: time, Length: 1072, dtype: float64

In [16]:
results = get_prediction_from_load(ds_test['trees'].values, ds_test[list_columns], ds_test['time'])

In [17]:
results

{'pred': [12.682406425476074,
  4.894477367401123,
  14.758463859558105,
  14.758463859558105,
  17.954404830932617,
  1.5937821865081787,
  14.758463859558105,
  1.5922574996948242,
  37.32883071899414,
  14.758463859558105,
  14.758505821228027,
  53.39189147949219,
  2.7885539531707764,
  14.75696849822998,
  14.759234428405762,
  14.758463859558105,
  44.680213928222656,
  4.89743709564209,
  1.6277093887329102,
  1.4970669746398926,
  14.758463859558105,
  1.5937821865081787,
  27.82819366455078,
  1.5922574996948242,
  36.944313049316406,
  13.64535903930664,
  2.7949717044830322,
  1.6255521774291992,
  37.03805160522461,
  14.758463859558105,
  14.758463859558105,
  84.39801025390625,
  1.5922574996948242,
  14.758463859558105,
  3.980473518371582,
  14.758463859558105,
  13.64535903930664,
  10.707067489624023,
  2.7885539531707764,
  4.077566146850586,
  6.100106716156006,
  39.093868255615234,
  2.839519739151001,
  14.758463859558105,
  24.007583618164062,
  14.758463859558

## Reinforcement Learning

In [18]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    #tree_format = TreeFormat_all(new_dicto,symbol)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
    reward -= pred['pred'][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    #new_tree = np.array([str(TreeFormat_all(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
        reward -= pred['pred'][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        if q_val[act[0]] == 0:
            q_val[act[0]] = reward
        else:
            q_val[act[0]] = max(reward,q_val[act[0]])
    return q_value_neo

def RL_max_available_actions(q_value, old_states, actions):
    available_actions = RL_available_actions(actions, old_states)
    index_av = [i[0] for i in available_actions]
    q_value_acotado = q_value[index_av[0]:index_av[-1]+1]
    return np.max(q_value_acotado)

In [19]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

----------RL DATA----------
shape ds_rl : (2000, 67)
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


## MULTI-VAL

##### Seleccionar data

In [20]:
min_num_actions = -1
buenos_idx = []
aaa = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        aaa.append(y_rl_ind)
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_idx",total_buenos_idx)
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree',
            #'history_old',
            #'history_new'
             ]
 
iteration = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree'
             ]

total_idx 2000


##### Parameters

In [21]:
#epsilon = 0.7 ## EPSILON
#lr_rl = 0.9 ## LEARNING RATE
#disc_fac = 0.999 ## DISCOUNT FACTOR
#epoch_rl = 200 ### TOTAL EPOCH POR DATA
#n = model_name ##SALT Nombre archivos

### Q - Learning - multival

##### Parameters

In [22]:
epsilon_ql = epsilon ## EPSILON
lr_ql = lr_rl ## LEARNING RATE
disc_fac_ql = disc_fac ## DISCOUNT FACTOR
epoch_ql = epoch_rl ### TOTAL EPOCH POR DATA

##### Loop

In [23]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)
            
            new_state = len(current_state)-1
            old_q_value = q_value_ql[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_ql[new_state,:], old_states_val, actions)
            temporal_difference = reward + disc_fac_ql * q_max - old_q_value
            #temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

Start: 2022-01-04 04:52:56.249516
2022-01-04 04:53:39.217720 : 10/2000, index 9 evaluating
2022-01-04 04:54:31.649405 : 20/2000, index 19 evaluating
2022-01-04 04:55:32.400290 : 30/2000, index 29 evaluating
2022-01-04 04:56:31.743683 : 40/2000, index 39 evaluating
2022-01-04 04:57:32.833182 : 50/2000, index 49 evaluating
2022-01-04 04:58:36.963116 : 60/2000, index 59 evaluating
2022-01-04 04:59:26.139159 : 70/2000, index 69 evaluating
2022-01-04 05:00:32.563308 : 80/2000, index 79 evaluating
2022-01-04 05:01:26.127720 : 90/2000, index 89 evaluating
2022-01-04 05:02:24.726411 : 100/2000, index 99 evaluating
2022-01-04 05:03:35.063862 : 110/2000, index 109 evaluating
2022-01-04 05:04:44.340862 : 120/2000, index 119 evaluating
2022-01-04 05:05:45.857745 : 130/2000, index 129 evaluating
2022-01-04 05:06:41.567332 : 140/2000, index 139 evaluating
2022-01-04 05:07:42.004022 : 150/2000, index 149 evaluating
2022-01-04 05:08:37.268732 : 160/2000, index 159 evaluating
2022-01-04 05:09:44.441779

2022-01-04 07:10:43.775887 : 1370/2000, index 1369 evaluating
2022-01-04 07:11:45.918206 : 1380/2000, index 1379 evaluating
2022-01-04 07:12:46.690100 : 1390/2000, index 1389 evaluating
2022-01-04 07:13:59.871549 : 1400/2000, index 1399 evaluating
2022-01-04 07:15:06.605943 : 1410/2000, index 1409 evaluating
2022-01-04 07:16:00.894336 : 1420/2000, index 1419 evaluating
2022-01-04 07:16:59.798210 : 1430/2000, index 1429 evaluating
2022-01-04 07:18:02.540927 : 1440/2000, index 1439 evaluating
2022-01-04 07:19:01.745986 : 1450/2000, index 1449 evaluating
2022-01-04 07:20:28.254954 : 1460/2000, index 1459 evaluating
2022-01-04 07:21:30.348764 : 1470/2000, index 1469 evaluating
2022-01-04 07:22:35.334582 : 1480/2000, index 1479 evaluating
2022-01-04 07:23:37.111425 : 1490/2000, index 1489 evaluating
2022-01-04 07:24:37.758849 : 1500/2000, index 1499 evaluating
2022-01-04 07:25:36.542661 : 1510/2000, index 1509 evaluating
2022-01-04 07:26:35.658611 : 1520/2000, index 1519 evaluating
2022-01-

In [24]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_iteration = ql_df[iteration]
ql_df_iteration.to_csv(URL + 'rl_csvs/ql_df_iteration'+str(n)+'.csv',index=False)

In [25]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final'+str(n)+'.csv',index=False)

In [26]:
ql_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.178822,78.982040,49.0,49.0,51.535479,898.922747,7.178822,29.982040,7.178822,29.982040,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.397385,56.175850,49.0,49.0,54.721300,51.492822,7.397385,7.175850,7.397385,7.175850,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.249290,56.580860,49.0,49.0,52.552212,57.469440,7.249290,7.580860,7.249290,7.580860,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.249290,56.217880,49.0,49.0,52.552212,52.097795,7.249290,7.217880,7.249290,7.217880,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.191776,56.240829,49.0,49.0,51.721646,52.429611,7.191776,7.240829,7.191776,7.240829,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.727798,30.594458,124.0,124.0,150.606931,8724.595346,12.272202,93.405542,12.272202,93.405542,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,109.797218,31.134521,124.0,124.0,201.719007,8623.997100,14.202782,92.865479,14.202782,92.865479,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,111.138603,30.568714,124.0,124.0,165.415527,8729.405177,12.861397,93.431286,12.861397,93.431286,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,108.463531,30.929544,124.0,124.0,241.381854,8662.109696,15.536469,93.070456,15.536469,93.070456,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [27]:
ql_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.237965,56.257210,49.0,49.0,52.388132,52.667094,7.237965,7.257210,7.237965,7.257210,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.614208,58.450417,46.0,46.0,159.118249,155.012873,12.614208,12.450417,12.614208,12.450417,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.232079,60.464760,48.0,48.0,149.623746,155.370238,12.232079,12.464760,12.232079,12.464760,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.843712,45.571594,48.0,48.0,4.649579,5.897155,2.156288,2.428406,2.156288,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.060776,111.837723,49.0,49.0,3851.539888,3948.579404,62.060776,62.837723,62.060776,62.837723,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.724228,81.469055,111.0,111.0,825.081269,872.076702,28.724228,29.530945,28.724228,29.530945,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1996,1996,199,103.812897,57.553539,122.0,122.0,330.770725,4153.346300,18.187103,64.446461,18.187103,64.446461,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.122696,71.872238,109.0,109.0,581.904459,1378.470699,24.122696,37.127762,24.122696,37.127762,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,157.501709,71.555191,136.0,136.0,462.323489,4153.133402,21.501709,64.444809,21.501709,64.444809,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## RL-Neo Multival

###### Parameters

In [28]:
epsilon_neo = epsilon ## EPSILON
epoch_neo = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [29]:
values_neo = []
values_final_neo = []
c_neo = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_neo = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_neo % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_neo}/{total_buenos_idx}, index {idx} evaluating')
    ############# ALGORITMO NEO #####################3
    for i in range(0,epoch_neo):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_neo,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_neo)    
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_neo,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_neo)
        q_value_neo = RLNeo_q_values(q_value_neo,current_state, reward)
        ##############################################################
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_neo,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]
        values_neo.append(row)
    values_final_neo.append(row)
    c_neo += 1
#print("values_neo ", values_neo)
print("len(values_neo[0])",len(values_neo[0]) )
print("len(values_neo)", len(values_neo))

Start: 2022-01-04 08:19:00.257137
2022-01-04 08:19:37.826851 : 10/2000, index 9 evaluating
2022-01-04 08:20:19.854744 : 20/2000, index 19 evaluating
2022-01-04 08:21:01.899168 : 30/2000, index 29 evaluating
2022-01-04 08:21:43.858297 : 40/2000, index 39 evaluating
2022-01-04 08:22:25.744009 : 50/2000, index 49 evaluating
2022-01-04 08:23:07.942358 : 60/2000, index 59 evaluating
2022-01-04 08:23:49.326424 : 70/2000, index 69 evaluating
2022-01-04 08:24:31.648540 : 80/2000, index 79 evaluating
2022-01-04 08:25:13.293666 : 90/2000, index 89 evaluating
2022-01-04 08:25:55.197771 : 100/2000, index 99 evaluating
2022-01-04 08:26:37.716698 : 110/2000, index 109 evaluating
2022-01-04 08:27:20.258950 : 120/2000, index 119 evaluating
2022-01-04 08:28:02.240470 : 130/2000, index 129 evaluating
2022-01-04 08:28:43.836956 : 140/2000, index 139 evaluating
2022-01-04 08:29:25.843048 : 150/2000, index 149 evaluating
2022-01-04 08:30:07.675504 : 160/2000, index 159 evaluating
2022-01-04 08:30:50.341480

2022-01-04 09:55:21.481988 : 1370/2000, index 1369 evaluating
2022-01-04 09:56:03.988853 : 1380/2000, index 1379 evaluating
2022-01-04 09:56:46.553967 : 1390/2000, index 1389 evaluating
2022-01-04 09:57:29.774858 : 1400/2000, index 1399 evaluating
2022-01-04 09:58:12.597783 : 1410/2000, index 1409 evaluating
2022-01-04 09:58:54.588897 : 1420/2000, index 1419 evaluating
2022-01-04 09:59:36.871092 : 1430/2000, index 1429 evaluating
2022-01-04 10:00:19.622573 : 1440/2000, index 1439 evaluating
2022-01-04 10:01:01.989531 : 1450/2000, index 1449 evaluating
2022-01-04 10:01:46.463540 : 1460/2000, index 1459 evaluating
2022-01-04 10:02:28.963205 : 1470/2000, index 1469 evaluating
2022-01-04 10:03:11.660946 : 1480/2000, index 1479 evaluating
2022-01-04 10:03:54.131494 : 1490/2000, index 1489 evaluating
2022-01-04 10:04:36.376067 : 1500/2000, index 1499 evaluating
2022-01-04 10:05:18.760169 : 1510/2000, index 1509 evaluating
2022-01-04 10:06:00.903102 : 1520/2000, index 1519 evaluating
2022-01-

In [30]:
neo_df = pd.DataFrame(values_neo,columns=rl_columns)
neo_df_iteration = neo_df[iteration]
neo_df_iteration.to_csv(URL + 'rl_csvs/neo_df_iteration'+str(n)+'.csv',index=False)

In [31]:
neo_df_final = pd.DataFrame(values_final_neo,columns=rl_columns)
neo_df_final.to_csv(URL + 'rl_csvs/neo_df_final'+str(n)+'.csv',index=False)

In [32]:
neo_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.181305,80.499557,49.0,49.0,51.571141,992.222122,7.181305,31.499557,7.181305,31.499557,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.284954,56.220333,49.0,49.0,53.070556,52.133210,7.284954,7.220333,7.284954,7.220333,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.509983,56.056423,49.0,49.0,56.399846,49.793108,7.509983,7.056423,7.509983,7.056423,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.174240,56.276321,49.0,49.0,51.469721,52.944853,7.174240,7.276321,7.174240,7.276321,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.249290,56.159523,49.0,49.0,52.552212,51.258770,7.249290,7.159523,7.249290,7.159523,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.462036,30.803637,124.0,124.0,157.200538,8685.562160,12.537964,93.196363,12.537964,93.196363,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.474747,30.959484,124.0,124.0,156.881970,8656.537599,12.525253,93.040516,12.525253,93.040516,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,109.612144,30.883787,124.0,124.0,207.010387,8670.629095,14.387856,93.116213,14.387856,93.116213,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,108.815323,30.896925,124.0,124.0,230.574419,8668.182580,15.184677,93.103075,15.184677,93.103075,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [33]:
neo_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.362774,56.178139,49.0,49.0,54.210439,51.525676,7.362774,7.178139,7.362774,7.178139,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,57.861134,58.093735,46.0,46.0,140.686490,146.258420,11.861134,12.093735,11.861134,12.093735,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,59.906139,60.637806,48.0,48.0,141.756155,159.714139,11.906139,12.637806,11.906139,12.637806,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.076698,94.374367,49.0,49.0,3853.516472,2058.833159,62.076698,45.374367,62.076698,45.374367,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P885;ᶲIRI_ID;...dire..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.423462,81.068062,111.0,111.0,807.893187,895.920923,28.423462,29.931938,28.423462,29.931938,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1996,1996,199,100.518944,58.968826,122.0,122.0,461.435776,3972.928859,21.481056,63.031174,21.481056,63.031174,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.567413,73.607254,109.0,109.0,603.557798,1252.646467,24.567413,35.392746,24.567413,35.392746,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.721390,72.638054,136.0,136.0,516.261553,4014.736214,22.721390,63.361946,22.721390,63.361946,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Sarsa Multival

###### Parameters

In [34]:
epsilon_sar = epsilon ## EPSILON
lr_sar = lr_rl ## LEARNING RATE
disc_fac_sar = disc_fac ## DISCOUNT FACTOR
epoch_sar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [35]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            new_state = len(current_state)-1
            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

Start: 2022-01-04 10:40:36.158627
2022-01-04 10:41:30.364116 : 10/2000, index 9 evaluating
2022-01-04 10:42:32.078243 : 20/2000, index 19 evaluating
2022-01-04 10:43:32.309523 : 30/2000, index 29 evaluating
2022-01-04 10:44:30.477842 : 40/2000, index 39 evaluating
2022-01-04 10:45:30.091644 : 50/2000, index 49 evaluating
2022-01-04 10:46:34.395100 : 60/2000, index 59 evaluating
2022-01-04 10:47:23.236422 : 70/2000, index 69 evaluating
2022-01-04 10:48:29.105398 : 80/2000, index 79 evaluating
2022-01-04 10:49:22.366806 : 90/2000, index 89 evaluating
2022-01-04 10:50:20.498637 : 100/2000, index 99 evaluating
2022-01-04 10:51:30.045714 : 110/2000, index 109 evaluating
2022-01-04 10:52:38.902291 : 120/2000, index 119 evaluating
2022-01-04 10:53:39.023557 : 130/2000, index 129 evaluating
2022-01-04 10:54:33.926782 : 140/2000, index 139 evaluating
2022-01-04 10:55:33.547096 : 150/2000, index 149 evaluating
2022-01-04 10:56:28.506898 : 160/2000, index 159 evaluating
2022-01-04 10:57:34.776400

2022-01-04 12:56:20.990565 : 1370/2000, index 1369 evaluating
2022-01-04 12:57:23.638847 : 1380/2000, index 1379 evaluating
2022-01-04 12:58:24.655857 : 1390/2000, index 1389 evaluating
2022-01-04 12:59:38.391661 : 1400/2000, index 1399 evaluating
2022-01-04 13:00:45.666726 : 1410/2000, index 1409 evaluating
2022-01-04 13:01:40.494930 : 1420/2000, index 1419 evaluating
2022-01-04 13:02:39.930592 : 1430/2000, index 1429 evaluating
2022-01-04 13:03:42.887730 : 1440/2000, index 1439 evaluating
2022-01-04 13:04:42.724206 : 1450/2000, index 1449 evaluating
2022-01-04 13:06:10.612407 : 1460/2000, index 1459 evaluating
2022-01-04 13:07:13.697354 : 1470/2000, index 1469 evaluating
2022-01-04 13:08:19.391197 : 1480/2000, index 1479 evaluating
2022-01-04 13:09:22.352984 : 1490/2000, index 1489 evaluating
2022-01-04 13:10:23.242792 : 1500/2000, index 1499 evaluating
2022-01-04 13:11:23.025147 : 1510/2000, index 1509 evaluating
2022-01-04 13:12:22.510581 : 1520/2000, index 1519 evaluating
2022-01-

In [36]:
sarsa_df = pd.DataFrame(values_sar,columns=rl_columns)
sarsa_df_iteration = sarsa_df[iteration]
sarsa_df_iteration.to_csv(URL + 'rl_csvs/sarsa_df_iteration'+str(n)+'.csv',index=False)

In [37]:
sarsa_df_final = pd.DataFrame(values_final_sar,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final'+str(n)+'.csv',index=False)

In [38]:
sarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.246452,79.526169,49.0,49.0,52.511071,931.846983,7.246452,30.526169,7.246452,30.526169,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.202168,81.146835,49.0,49.0,51.871217,1033.419022,7.202168,32.146835,7.202168,32.146835,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.178329,79.491386,49.0,49.0,51.528414,929.724645,7.178329,30.491386,7.178329,30.491386,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.195457,81.916756,49.0,49.0,51.774608,1083.512804,7.195457,32.916756,7.195457,32.916756,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
4,0,4,56.255432,56.178276,49.0,49.0,52.641295,51.527647,7.255432,7.178276,7.255432,7.178276,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.664658,31.067753,124.0,124.0,152.160672,8636.402563,12.335342,92.932247,12.335342,92.932247,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.348137,31.201813,124.0,124.0,160.069640,8611.503558,12.651863,92.798187,12.651863,92.798187,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,111.393791,30.851292,124.0,124.0,158.916500,8676.681866,12.606209,93.148708,12.606209,93.148708,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,109.263954,30.635069,124.0,124.0,217.151047,8717.010361,14.736046,93.364931,14.736046,93.364931,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [39]:
sarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.236160,56.249290,49.0,49.0,52.362016,52.552212,7.236160,7.249290,7.236160,7.249290,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.196507,58.367386,46.0,46.0,148.754771,152.952233,12.196507,12.367386,12.196507,12.367386,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.399261,59.888920,48.0,48.0,153.741685,141.346415,12.399261,11.888920,12.399261,11.888920,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,112.009010,110.931374,49.0,49.0,3970.135381,3835.495036,63.009010,61.931374,63.009010,61.931374,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.650909,139.485413,111.0,111.0,820.874611,811.418731,28.650909,28.485413,28.650909,28.485413,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,99.742996,58.792938,122.0,122.0,495.374217,3995.132657,22.257004,63.207062,22.257004,63.207062,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,132.056305,73.594482,109.0,109.0,531.593197,1253.550675,23.056305,35.405518,23.056305,35.405518,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.145218,71.542633,136.0,136.0,490.410676,4154.752153,22.145218,64.457367,22.145218,64.457367,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Expected-Sarsa Multival

###### Parameters

In [40]:
epsilon_exsar = epsilon ## EPSILON
epsilon_exsar_alg = epsilon
lr_exsar = lr_rl ## LEARNING RATE
disc_fac_exsar = disc_fac ## DISCOUNT FACTOR
epoch_exsar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [41]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_exsar[new_state,:], old_states_val, actions)
            #q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar_alg/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar_alg)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

Start: 2022-01-04 14:05:07.254905
2022-01-04 14:06:01.183852 : 10/2000, index 9 evaluating
2022-01-04 14:07:02.144739 : 20/2000, index 19 evaluating
2022-01-04 14:08:02.213518 : 30/2000, index 29 evaluating
2022-01-04 14:09:00.366061 : 40/2000, index 39 evaluating
2022-01-04 14:10:00.091522 : 50/2000, index 49 evaluating
2022-01-04 14:11:04.405727 : 60/2000, index 59 evaluating
2022-01-04 14:11:53.362516 : 70/2000, index 69 evaluating
2022-01-04 14:12:59.183611 : 80/2000, index 79 evaluating
2022-01-04 14:13:52.704443 : 90/2000, index 89 evaluating
2022-01-04 14:14:50.814517 : 100/2000, index 99 evaluating
2022-01-04 14:16:00.728106 : 110/2000, index 109 evaluating
2022-01-04 14:17:09.905236 : 120/2000, index 119 evaluating
2022-01-04 14:18:10.604887 : 130/2000, index 129 evaluating
2022-01-04 14:19:05.625461 : 140/2000, index 139 evaluating
2022-01-04 14:20:05.686299 : 150/2000, index 149 evaluating
2022-01-04 14:21:00.595365 : 160/2000, index 159 evaluating
2022-01-04 14:22:07.056649

2022-01-04 16:21:21.278608 : 1370/2000, index 1369 evaluating
2022-01-04 16:22:24.532723 : 1380/2000, index 1379 evaluating
2022-01-04 16:23:25.968661 : 1390/2000, index 1389 evaluating
2022-01-04 16:24:40.301259 : 1400/2000, index 1399 evaluating
2022-01-04 16:25:48.102414 : 1410/2000, index 1409 evaluating
2022-01-04 16:26:43.354194 : 1420/2000, index 1419 evaluating
2022-01-04 16:27:43.235653 : 1430/2000, index 1429 evaluating
2022-01-04 16:28:46.761115 : 1440/2000, index 1439 evaluating
2022-01-04 16:29:46.845868 : 1450/2000, index 1449 evaluating
2022-01-04 16:31:14.991784 : 1460/2000, index 1459 evaluating
2022-01-04 16:32:18.130177 : 1470/2000, index 1469 evaluating
2022-01-04 16:33:24.337613 : 1480/2000, index 1479 evaluating
2022-01-04 16:34:27.567266 : 1490/2000, index 1489 evaluating
2022-01-04 16:35:29.281706 : 1500/2000, index 1499 evaluating
2022-01-04 16:36:29.015423 : 1510/2000, index 1509 evaluating
2022-01-04 16:37:28.993495 : 1520/2000, index 1519 evaluating
2022-01-

In [42]:
exsarsa_df = pd.DataFrame(values_exsar,columns=rl_columns)
exsarsa_df_iteration = exsarsa_df[iteration]
exsarsa_df_iteration.to_csv(URL + 'rl_csvs/exsarsa_df_iteration'+str(n)+'.csv',index=False)

In [43]:
exsarsa_df_final = pd.DataFrame(values_final_exsar,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final'+str(n)+'.csv',index=False)

In [44]:
exsarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.239712,56.296265,49.0,49.0,52.413426,53.235478,7.239712,7.296265,7.239712,7.296265,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,0,1,56.212204,56.194420,49.0,49.0,52.015886,51.759677,7.212204,7.194420,7.212204,7.194420,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.235561,56.280640,49.0,49.0,52.353348,53.007714,7.235561,7.280640,7.235561,7.280640,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.175224,56.175198,49.0,49.0,51.483844,51.483461,7.175224,7.175198,7.175224,7.175198,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.210648,56.620220,49.0,49.0,51.993439,58.067756,7.210648,7.620220,7.210648,7.620220,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.457909,30.608841,124.0,124.0,157.304056,8721.908590,12.542091,93.391159,12.542091,93.391159,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,109.525253,30.431150,124.0,124.0,209.518292,8755.129609,14.474747,93.568850,14.474747,93.568850,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.298302,30.461597,124.0,124.0,187.736536,8749.432753,13.701698,93.538403,13.701698,93.538403,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,110.989037,30.827759,124.0,124.0,169.285170,8681.066532,13.010963,93.172241,13.010963,93.172241,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [45]:
exsarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.226719,56.177376,49.0,49.0,52.225466,51.514723,7.226719,7.177376,7.226719,7.177376,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.578320,58.542988,46.0,46.0,158.214123,157.326544,12.578320,12.542988,12.578320,12.542988,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.831768,60.183807,48.0,48.0,164.654271,148.445162,12.831768,12.183807,12.831768,12.183807,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,44.709270,45.859173,48.0,48.0,10.828901,4.583141,3.290730,2.140827,3.290730,2.140827,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.484993,111.015686,49.0,49.0,3904.374348,3845.945314,62.484993,62.015686,62.484993,62.015686,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,138.609665,139.725708,111.0,111.0,762.293597,825.166301,27.609665,28.725708,27.609665,28.725708,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,100.653809,59.309273,122.0,122.0,455.659888,3930.127281,21.346191,62.690727,21.346191,62.690727,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.476517,71.816002,109.0,109.0,599.099871,1382.649715,24.476517,37.183998,24.476517,37.183998,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,157.731415,70.920532,136.0,136.0,472.254389,4235.337126,21.731415,65.079468,21.731415,65.079468,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


In [46]:
1+1

2

### Archivos enteros

In [47]:
import os
path = "/home/school/math/final_analysis.csv"

In [48]:
os.remove(URL + 'rl_csvs/ql_df.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/ccarmona/memoria/dataset/rl_csvs/ql_df.csv'

In [ ]:
os.remove(URL + 'rl_csvs/sarsa_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/neo_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/exsarsa_df.csv')

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/sarsa_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/exsarsa_df.csv',index=False)